In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
# from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [ ]:
## create map
Map = geemap.Map(height="400px")
#Map.set_options('HYBRID');
Map

In [ ]:
# read HAND data to generate drainage paths 
hand30_100 = ee.ImageCollection('users/gena/global-hand/hand-100')
hand30_1000 = ee.Image('users/gena/GlobalHAND/30m/hand-1000')
hand30_5000 = ee.Image('users/gena/GlobalHAND/30m/hand-5000')
srtm = ee.Image("USGS/SRTMGL1_003")

# Map.addLayer(hand30_5000,{},"hand",False)

## exclude SWBD water
swbd = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask')
swbdMask = swbd.unmask().Not().focal_median(1) #.focal_max(5) #.add(0.2);

## select HAND height by flow accumuation levels
thresh = 0
threshStr=str(thresh)
FlowAccu = hand30_1000 # options: hand30_100, hand30_1000, hand30_5000
FA = 1000 # options: 100, 1000, 5000
FAstr=str(FA)
HANDmFA = FlowAccu.lte(thresh).focal_max(1).focal_mode(2, 'circle', 'pixels', 5).mask(swbdMask)
HANDmFA = HANDmFA.mask(HANDmFA)
HANDscale =HANDmFA.projection().nominalScale()
HANDCRS =HANDmFA.projection().crs()
Map.addLayer(HANDmFA, {'palette':['0020ff'], 'opacity':0.8},'water (HAND < '+threshStr+'m, FA'+FAstr+')', False);


In [ ]:
# read and map HydroSHEDS river data
dataset = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers');
# Paint "RIV_ORD" (river order) value to an image for visualization.
datasetVis = ee.Image().byte().paint(dataset, 'RIV_ORD', 2);
visParams = {'min': 1,'max': 10,'palette': ['08519c', '3182bd', '6baed6', 'bdd7e7', 'eff3ff']};

# Map.addLayer(datasetVis, visParams, 'Free flowing rivers');
#Map.addLayer(dataset, {}, 'Free flowing rivers', False);

In [ ]:
# Read surface water occurance
water = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select(['occurrence']).gte(50);
# water = water.reproject(**{'crs':HANDCRS,'scale':HANDscale})##'crsTransform':esa_PROJ.transform,

#visualization = {'bands': ['occurrence'],'min': 50,'max': 100,'palette':['black'] };['ffffff', 'ffbbbb', '0000ff']
# Map.addLayer(water, {'palette':['black']}, 'JRC Water occurrence',False);
#var watermask = water.select('occurrence').gte(90).where(water.select('occurrence'),0).unmask(1).selfMask()


In [ ]:
#Create combined water layer and mask
#combWater = ee.ImageCollection([HANDmFA,water]).mosaic()
combWater = HANDmFA.blend(water).gt(0)
combWaterScale =combWater.projection().nominalScale()
combWaterCRS =combWater.projection().crs()

Map.addLayer(combWater, {'palette':['blue']}, 'Combined water',False);

In [ ]:
# Buffer waterways by riparian zone definitions 
Distance = combWater.unmask().distance(ee.Kernel.euclidean(200, 'meters'))
halfpixel = combWaterScale.multiply(0.5)
## https://doi.org/10.1016/j.jenvman.2019.109391
nutrientBuffer = Distance.updateMask(Distance.lte(ee.Number(3).subtract(halfpixel)))#.updateMask(Distance.gt(0))
floraBuffer = Distance.updateMask(Distance.lte(ee.Number(24).subtract(halfpixel)))#.updateMask(Distance.gt(0))
birdBuffer = Distance.updateMask(Distance.lte(ee.Number(144).subtract(halfpixel)))#.updateMask(Distance.gt(0))
riparianBuffer = birdBuffer
riparianMask = riparianBuffer.updateMask(riparianBuffer.gt(0))
# riparianMask = riparianMask.reproject(**{'crs':greenCRS,'scale':greenScale})##'crsTransform':esa_PROJ.transform,

# Map.addLayer(Distance,{'min': 0, 'max': 200},"Distance",False)
#Map.addLayer(nutrientBuffer,{'min': 0, 'max': 200},"nutrientBuffer")
#Map.addLayer(floraBuffer,{'min': 0, 'max': 200},"floraBuffer")
#Map.addLayer(riparianBuffer,{'min': 0, 'max': 200},"riparianBuffer",False)
Map.addLayer(riparianMask,{'min': 0, 'max': 200},"riparianMask")

In [ ]:
## Calculate and load vegetation and water cover rasters

NDVIthreshold = 0.4 # decimal
NDWIthreshold = 0.3 # decimal
year = 2020

yearStr = str(year)
startdate = ''+yearStr+'-01-01'
enddate = ''+yearStr+'-12-31'

s2 = ee.ImageCollection("COPERNICUS/S2")

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDWI(image):
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
  return image.addBands(ndwi)

greenblue = s2.filterDate(startdate, enddate).map(addNDVI).map(addNDWI)
green = greenblue.qualityMosaic('NDVI').select('NDVI').float();
blue = greenblue.qualityMosaic('NDWI').select('NDWI').float();

greenScale = green.projection().nominalScale()
greenCRS = green.projection().crs()
#green = green.addBands(ee.Image(year).rename('time_start'))
greenmask = green.updateMask(green.select('NDVI').gte(NDVIthreshold))
bluenmask = blue.updateMask(blue.select('NDWI').gte(NDWIthreshold))
greenbluemask = greenmask.blend(bluenmask).rename('greenblue')
# greenbluemask = greenbluemask.reproject(**{'crs':combWaterCRS,'scale':combWaterScale})##'crsTransform':esa_PROJ.transform,

# Map.addLayer(greenmask,{},"green areas",False)
Map.addLayer(greenbluemask,{},"green/blue areas",False)


In [ ]:
#mask green and blue pixels to riparian areas
ripariangreenmask = greenmask.updateMask(riparianMask)
# Map.addLayer(ripariangreenmask,{},"green riparian areas")

ripariangreenbluemask = greenbluemask.updateMask(riparianMask)
Map.addLayer(ripariangreenbluemask,{},"green/blue riparian areas")

# Compute indicator

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

In [ ]:
this_indicatorDF = pd.DataFrame() 
this_indicator = ee.FeatureCollection([])
this_indicator

In [ ]:
# function to convert pixel counts to area percentages and save to FC as property
def toPct(feat):
    pctEq = ee.Number(1).subtract(((feat.getNumber('RiparianBlueGreenPixels')).divide(feat.getNumber('RiparianPixels'))))
    pct = ee.Algorithms.If(
        feat.getNumber('RiparianPixels').eq(0),
        'NA',
        ee.Algorithms.If(
            pctEq.lte(0),
            0,
            pctEq)
    )
    return feat.set({
        'PctRiparianwoBlueGreen_'+yearStr+'': pct
  })

In [ ]:
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcsDF(FC,DF):
    # reduce images to get vegetation and built-up pixel counts
    pixelcounts = ripariangreenbluemask.reduceRegions(FC,ee.Reducer.count().setOutputs(['RiparianBlueGreenPixels']),combWaterScale)
    pixelcounts = riparianMask.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['RiparianPixels']),combWaterScale)

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','RiparianBlueGreenPixels','RiparianPixels','PctRiparianwoBlueGreen_'+yearStr+''])

    # store in df and append
    df = geemap.ee_to_pandas(pixelcounts)
    df = df.rename(columns={'PctRiparianwoBlueGreen_'+yearStr+'': 'GRE_4_6_percentRiparianZonewoVegorWatercover'+yearStr+''})
    DF = DF.append(df)
    return DF


In [ ]:
# use if using EE featurecollections for calculations
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcsEE(FC,DF):
    # reduce images to get vegetation and built-up pixel counts
    pixelcounts = ripariangreenbluemask.reduceRegions(FC,ee.Reducer.count().setOutputs(['RiparianBlueGreenPixels']),combWaterScale)
    pixelcounts = riparianMask.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['RiparianPixels']),combWaterScale)

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctRiparianwoBlueGreen_'+yearStr+''])
    
    # amend existing FeatureCollection with pixel counts for new geographies
    DF = ee.FeatureCollection([DF,pixelcounts]).flatten()
    return DF


In [ ]:
for i in range(0,len(boundary_georef)): #9,10):# for Mexico City
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicatorDF = CountCalcsDF(boundary_geo_ee,this_indicatorDF)
    # this_indicator = CountCalcsEE(boundary_geo_ee,this_indicator) # run this instead if using CountCalcsEE approach


    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicatorDF = CountCalcsDF(boundary_geo_ee,this_indicatorDF)
    # this_indicator = CountCalcsEE(boundary_geo_ee,this_indicator) # run this instead if using CountCalcsEE approach


In [ ]:
this_indicatorDF#.head(n=50)

# Workaround for timeout problems for specific geographies

In [ ]:
    # if timeout problems for a geography, use CountCalcsEE and save ee.FeatureCollection as EE asset before coverting to dataframe
    task = ee.batch.Export.table.toAsset(
        collection = this_indicator, 
        description = 'thisindicator',
        assetId = 'users/emackres/thisindicator',
    )

    task.start()

In [ ]:
    task.status()

In [ ]:
    # wait until EE asset is generated (with task status of "COMPLETED") before running
    # store FC in df and apend to this_indicatorDF

    FC = ee.FeatureCollection('users/emackres/thisindicator')
    df = geemap.ee_to_pandas(FC)
    df = df.rename(columns={'PctRiparianwoBlueGreen_'+yearStr+'': 'GRE_4_6_percentRiparianZonewoVegorWatercover'+yearStr+''})
    this_indicatorDF = this_indicatorDF.append(df)


In [ ]:
    this_indicatorDF

In [ ]:
    #delete GEE asset 
    ee.data.deleteAsset('users/emackres/thisindicator')

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2test.csv') 
cities_indicators#.head()

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicator")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicatorDF,
                                            indicator_name = 'GRE_4_6_percentRiparianZonewoVegorWatercover'+yearStr+'')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2test.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')